# Classes

A `class` is the foundation of object oriented program. They are 'containers' that hold methods and/ or data. You have already interacted with classes in python whether you knew it or not. 

An example of a class is a `numpy.array`. In the case of the `numpy.array`, the data being stored is lists of numbers, strings, etc., and the methods are functions like `numpy.array().resize()` or `numpy.array().argmax()`.

In [6]:
import numpy as np

a = np.array([1,2,3,4])  # a is an object of type numpy.array, numpy.array is the class

a.resize(2,2)  # .resize() is a method of the numpy.array class

## What data can be stored in classes? 

Any form of data you can think of can be stored in classes. Strings, dictionaries, other classes, etc... It's all up to you!

## What are class methods?

Class methods are functions that are 'owned' by the class. A method is a function that touches the data of the class in some way, whether this is changing the data or simply returning the data. 

Enough chit chat, lets get down to business. Here is the minimal definition on how you define a class.

In [45]:
class myobject:  #creates a class called 'myobject'
    def __init__(self):  #the initializer of the class, the class needs to know about itself to be initialized
        return

a = myobject()  #instantiates variable 'a' as type 'myobject'

`myobject` is the name of the class, and has a single member function called `__init__`. `__init__` is a required method in every class, as it tells python how to construct the class. If you want the class to have a member variable, aka data, you would instantiate that variable in the `__init__` method. Every class method (except for `@staticmethod` but we will get into that later) must have the argument `self`. This tells the member function about its parent class, allowing the function to access the class's data. 

Lets add some member variables!

In [48]:
class myobject:
    def __init__(self, arg1):  #you can pass arguments to the initializer
        self.variable1 = arg1  #to assign a member variable, you must do self.variable
        self.variable2 = 'hello'
        self.variable3 = {'one':1}
        return

a = myobject(1)

To access the member variables, we use similair notation as in the initializer:

In [51]:
print(f'variable1 is {a.variable1}')

variable1 is 1


Now its time for member functions. Member functions can be declared in multiple different ways, let us first start with the simplest.

In [55]:
class myobject:
    def __init__(self):
        self.variable = 'Hello'
        return
    def print(self, arg):
        print(f'{self.variable}, {arg}')
        
a = myobject()
a.print('something')

Hello, something


You can also add methods, called dunder or magic methods, for 'operator overloading'. A helpful use case of dunder methods is the `__call__` method, which allows you to create something called a Functor. A functor is a mix between a class and a function, which allows a class object to behave like a function, i.e. you can instantiate a function with whatever inner-workings you want (helpful for material properties...). A simple example of a functor is an addition functor. 

Say you want to add two numbers together, a + b. Lets also say that a is some constant that may change, for example you want to add 4 to b, no matter what b is, but you want to be able to change 4, 3, 17, 9, etc. To do this with a typical function, you could either define a new function for adding 3 or adding 4 or so on, or you could pass a as an argument. Alternatively, you could use a functor, and initialize the function with whatever number you want. Lets look at the second option, the functor:

In [10]:
class add:
    def __init__(self, a):
        self.a = a
        return

    def __call__(self,b):
        return b + self.a

Now, if I have the liberty to set a to be whatever I want without having to create new functions each time or pass it as an argument:

In [12]:
# set a to 42:
add42 = add(42)
print(add42(5))

# set a to 13:
add13 = add(13)
print(add13(5))

47
18


Now obviously this is a bit of a silly and over complex approach to a problem like this, as passing a as an argument to a function is not too difficult. However, what about for a problem such as material properties like the diffusion coefficient?

$$ \frac{2D_i}{h^2} \cdot \biggr[ -\frac{D_{i+1}}{D_i + D_{i+1}}\phi_{i+1} +\biggr( \frac{D_{i+1}}{D_i + D_{i+1}} + \frac{D_{i-1}}{D_i + D_{i-1}}  \biggr)\phi_i - \frac{D_{i-1}}{D_i + D_{i-1}}\phi_{i-1} \biggr] $$

You could create a functor object, that to initialize would take the diffusion coefficients for each boundary and the physical bounds of the problem, would take (as arguments to the functor) the mesh spacing and the location you want, and would return the left ($i-1$), center ($i$), and right ($i+1$).

What benefit does a functor have over a simple function that takes all the same things? 

1. Can initialize right at the beginning of the problem and you wouldn't have to worry about passing correct arguments later
2. Much simpler to debug than a classic python function, no copy and paste required, and because the functor gets initialized prior to the problem you do not have to worry about reconstructing the exact conditions the function is seeing to determine what is going wrong
3. Would require minimal (likely no) rewriting to go from problem to problem, as the difference are taken into account by the functor's initialization

Some sample code to get you started on making your functor is below!

In [5]:
class NumericalDiffCoeff():
    def __init__(self, inputs):

        #set your parameters here!

        self.replace_me = inputs # change me

        return

    def whichDiff(self, x):
        # return the diffusion coefficient at x
        return 

    def left(self, param):
        #This function to find the left (i-1) diffusion coefficient
        return 

    def right(self, param):
        #This function to find the right (i-1) diffusion coefficient
        return 

    def center(self, param):
        #This function to find the center (i-1) diffusion coefficient
        return

    def __call__(self, param):
        #This function is NOT optimized, see if you can play around with it...
        
        left = self.left(param)
        right = self.right(param)
        center = self.center(param)

        # returns a list of the diffusion coefficients for a single row
        # could use this in conjunction with np.array splicing to fill in one fell swoop...
        return [left, center, right]

    def __getitem__(self,param):
        #This function overrides indexing, the idea being you index the diffusion coeff at x
        return self.whichDiff(param)

#how to initialize
NumDCoeff = NumericalDiffCoeff(inputs=None)

# to call at some x location 
x = 0
NumDCoeff(x)

# to index at some x location
NumDCoeff[x]